In [6]:
import random
import requests
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import re
import unicodedata


In [ ]:
%pip install pandas
def generate_variations(word,language='ancient greek',show=False):
        # CREATES A FOLDER STRUCTURE THAT CONTAINS WORDS ORGANIZED BY PART OF SPEECH
        #DECLINES WORDS
        #test words
        #'Ῥωμαῖος' romans
        #'μισώ' hate
        #'εἰμί' are to be
        #word = word.lower()
        #https://en.wiktionary.org/wiki/Ῥωμαῖος
        url= requests.get(f'https://en.wiktionary.org/wiki/{quote(word)}')
        soup = bs(url.text, 'html.parser')




        #remove unwanted classes
        for tag in soup.find_all('span',{'class':'tr Latn'}):
                tag.decompose()
        for tag in soup.find_all('table',{'class':'audiotable'}):
                tag.decompose()
        for tag in soup.find_all('sup'):
                tag.decompose()

        #HEADERS ---------------------------
        #find all tables
        tables =soup.find_all('table')
        part_of_speech_h3 =[]
        h4 =[] # inflection/declension
        nav=[] #verb tense, noun first second third delcension
        gender =None

        #DATAFRAMES----------------------------------

        dataframes = pd.read_html(soup.prettify())
        for i,df in enumerate(dataframes):
                
                header = tables[i].find_previous('h2').text.replace('[edit]','').lower().strip()

                #GENDER
                gender = tables[i].find_previous(class_="gender")
                if(gender != None):
                        gender = gender.find_next("abbr").text
                elif (gender ==None):
                        gender = "n"


                #if(header =='greek'):
                        # if modern greek ignore
                        #header = ''
                #if(header =='ancient greek'):
                        #header = 'greek'
                if(header != language):
                        part_of_speech_h3.append('')
                        h4.append('')
                        nav.append('')
                        continue
                if(tables[i].find_previous('h3')):
                        part_of_speech_h3.append(tables[i].find_previous('h3').text
                        .replace('[edit]','')
                        .replace('\"','').lower().strip())
                else:
                        part_of_speech_h3.append('')
                if(tables[i].find_previous('h4')):
                        h4.append(tables[i].find_previous('h4').text
                        .replace('[edit]','')
                        .replace('\"','').lower().strip())
                else:
                        h4.append('')
                if(tables[i].find_previous('div',{'class':'NavHead'})):
                        nav.append(tables[i].find_previous('div',{'class':'NavHead'}).text
                        .replace('hide ▲','')
                        .replace('\"','').replace('/','-').replace(':','-').strip().lower())
                        #  : / mess up the file path so you need to remove them
                else:
                        nav.append('')
                ##display(df)
                ##CREATE A FOLDER
                root_dir = f'{header.replace(" ", "_")}_variations'
                os.makedirs(f'{root_dir}/{part_of_speech_h3[i]}/{word.lower()}/',exist_ok=True)
                print(f'{root_dir}/{part_of_speech_h3[i]}/{word}/')
                ##os.makedirs(nav[i],exist_ok=True)

                #print(h3[i])
                #print(h4[i])
                #print(nav[i])
                #print('WRITE TO FILE--------------------------------')

                df=dataframes[i]

             
 
                # for appending df2 at the end of df1
                df.loc[len(df)]= gender
                df.iat[-1,0] = "gender"

                df=df.dropna(how='all',axis=0)



                df=df.replace({'➤':''}, regex=True)
                df=df.replace({'•':''}, regex=True)
                df=df.replace({'\[':'','\]':'','\{':'','\}':''}, regex=True)
                df=df.replace({'—':' '}, regex=True)
                #df=df.replace({'/':' ',r'\\':' '}, regex=True)
                df=df.replace(r"\([^)]*\)","",regex=True)



                # make lowercase
                df = df.applymap(lambda s: s.lower() if type(s) == str else s)

                #df = df.applymap(lambda s:strip_accents_and_lowercase(s) if type(s) == str else s)
                df = df.applymap(lambda s: s.split(',', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('-', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('+', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('/', 1)[0] if type(s) == str else s)
                df = df.applymap(lambda s: s.split('\\', 1)[0] if type(s) == str else s)

                df= df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


                df = df.replace(r"\s\s+"," ",regex=True)
              
                # NAN ARE REPALCED BY SPACE CHARACTER
                df.to_csv(f'{root_dir}/{part_of_speech_h3[i]}/{word.lower()}/{nav[i]}_{h4[i]}.csv',index=False)

                return df

Note: you may need to restart the kernel to use updated packages.


In [ ]:
def 